<a href="https://colab.research.google.com/github/Ilyaas189/CE888-Lab_Work/blob/main/Lab_08_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
pip install --upgrade kaggle

In [ ]:
#import os
#os.environ['KAGGLE_USERNAME'] = "ilyas189" # username from the json file
#os.environ['KAGGLE_KEY'] = "e9a5f91ca5c7a9eb3b160650cdc095d3" # key from the json file
#!kaggle datasets download 'catsdogshorses-and-humans' # api copied from kaggle (https://www.kaggle.com/c/dogs-vs-cats/data)

**Uploading the data (Zip file)**

In [ ]:
import os
from zipfile import ZipFile
file_name = "/content/data.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')



# # Get all the paths
# data_dir_list = os.listdir('/content/data.zip')
# #print(data_dir_list)
# path, dirs, files = next(os.walk("/content/data.zip"))
# file_count = len(files)
# print(file_count)

**MAking the New Base Directory**

In [ ]:
# Make new base directory
original_dataset_dir = '/content/data'
base_dir = '/content/cats_and_dogs_horse_humans_Final'
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

**Randomly splitting the data for Test, Train and VAlidation**

In [ ]:
import shutil
fnames = ['cat.{}.jpg'.format(i) for i in range(1,100)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'cats', fname)
    dst = os.path.join(train_dir, fname)
    # print(src,dst)
    shutil.copyfile(src, dst)
    
fnames = ['cat.{}.jpg'.format(i) for i in range(100, 150)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'cats', fname)
    dst = os.path.join(validation_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['cat.{}.jpg'.format(i) for i in range(150, 202)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'cats', fname)
    dst = os.path.join(test_dir, fname)
    shutil.copyfile(src, dst)
    
fnames = ['dog.{}.jpg'.format(i) for i in range(1,100)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'dogs', fname)
    dst = os.path.join(train_dir, fname)
    shutil.copyfile(src, dst)
#
fnames = ['dog.{}.jpg'.format(i) for i in range(100, 150)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'dogs', fname)
    dst = os.path.join(validation_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(150, 202)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'dogs', fname)
    dst = os.path.join(test_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['horse-{}.jpg'.format(i) for i in range(1,100)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'horses', fname)
    dst = os.path.join(train_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['horse-{}.jpg'.format(i) for i in range(100, 150)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'horses', fname)
    dst = os.path.join(validation_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['horse-{}.jpg'.format(i) for i in range(150, 202)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'horses', fname)
    dst = os.path.join(test_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['rider-{}.jpg'.format(i) for i in range(1,100)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'Humans', fname)
    dst = os.path.join(train_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['rider-{}.jpg'.format(i) for i in range(100, 150)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'Humans', fname)
    dst = os.path.join(validation_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['rider-{}.jpg'.format(i) for i in range(150, 202)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,'Humans', fname)
    dst = os.path.join(test_dir, fname)
    shutil.copyfile(src, dst)
print('done')

** and learning rate**

In [ ]:
import numpy as np
from random import shuffle

IMG_SIZE = 224
LR = 1e-4

**To return Image Labelling**

In [ ]:
def label_img(img):
    word_label = img.split('.')[0]
    if word_label == 'dog': return 1
    elif word_label == 'cat': return 0
    else:
      word_label = img.split('-')[0]
      if word_label == 'horse': return 2
      elif word_label == 'rider': return 3

Splitting the data

In [ ]:
import cv2
def createDataSplitSet(datapath):
    X=[]
    y=[]
    for img in os.listdir(datapath):
        label = label_img(img)
        # print(label)
        path = os.path.join(datapath, img)
        image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        X.append(np.array(image))
        y.append(label)
    return np.array(X), np.array(y)

**Employing the function of data splitting**

In [ ]:
train_X, train_y = createDataSplitSet(train_dir)
val_X, val_y = createDataSplitSet(validation_dir)
test_X, test_y = createDataSplitSet(test_dir)
print(train_X.shape)
print(train_y)

print(val_X.shape)
print(test_X.shape)

**Importing the VGG19 Model**

In [ ]:
## VGG 19

from tensorflow.keras.applications import VGG19
# from tensorflow.keras.applications import yolo
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model


img_input = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

**Initializing the VGG19 model**

In [ ]:
model = VGG19(
    include_top=True,
    weights="imagenet",
    input_tensor=img_input,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


model.summary()

**Fine Tuning the Model **

In [ ]:
last_layer = model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(64, activation='relu', name='fc2')(x)
out = Dense(4, activation='sigmoid', name='output')(x)  ## 4 classes
model = Model(img_input, out)

for layer in model.layers[:-3]:
	layer.trainable = False

model.summary()

**Set Loss Function and Optimizer**

In [ ]:
from keras.utils.np_utils import to_categorical
target=to_categorical(train_y)
val_target=to_categorical(val_y)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg19_model.h5', save_best_only=True)
]

**Training the Model**

In [ ]:
history = model.fit(train_X,target,
                              batch_size=20,
                              epochs=20, 
                              validation_data=(val_X, val_target),
                              callbacks=my_callbacks)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
test_target=to_categorical(test_y)
## Test Accuracy
predictions = model.predict(test_X)
ypred = predictions > 0.5
test_acc = accuracy_score(test_target, ypred)
print('done')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

precision, recall, f1score, _ = precision_recall_fscore_support(test_target, ypred, average='weighted')

auc = roc_auc_score(test_target, ypred)

print("Train Accuracy:\t", acc[-1])
print("Val Accuracy:\t", val_acc[-1])
print("Test Accuracy:\t", test_acc)
print("Precision:\t", precision)
print("Recall:\t\t", recall)
print("F1 Score:\t", f1score)
print("AUC:\t\t", auc)